In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
from matplotlib import pyplot as plt
import cv2
import gc
import tensorflow as tf
from tqdm import tqdm
from keras.models import Sequential
from keras import layers
from keras.optimizers import Adam
from keras.applications import DenseNet121,DenseNet169,DenseNet201

In [2]:
# connecting drive
from google.colab import drive
drive.mount('/content/drive')

# change to project directory
%cd /content/drive/My\ Drive/AI4ALL\ Diabetic\ Retinopathy\ Research/Early\ ML\ Model/aptos2019-blindness-detection

Mounted at /content/drive
/content/drive/.shortcut-targets-by-id/1DwcsYZWFxUskq7gNIgNqPDzvPfHJSdsh/AI4ALL Diabetic Retinopathy Research/Early ML Model/aptos2019-blindness-detection


In [3]:
x_train1 = np.load('x_train1.npy')
y_train1 = np.load('y_train1.npy')

In [4]:
from sklearn.model_selection import train_test_split
x_sptrain, x_spval, y_sptrain, y_spval = train_test_split(
    x_train1, y_train1,
    random_state=42
)
print("train-validation splitted ...")

train-validation splitted ...


In [5]:
del x_train1,y_train1
gc.collect()

43

In [40]:
y_sptrain

array([[ True,  True,  True, False, False],
       [ True, False, False, False, False],
       [ True,  True,  True,  True,  True],
       ...,
       [ True,  True,  True, False, False],
       [ True,  True, False, False, False],
       [ True, False, False, False, False]])

In [51]:
from keras.preprocessing.image import ImageDataGenerator
def create_datagen():
    return ImageDataGenerator(
         horizontal_flip=True,   # randomly flip images
         vertical_flip=True,     # randomly flip images
         rotation_range=20 ,      # Degree range for random rotations
         zoom_range=0.1

    )

# Using original generator
data_generator = create_datagen().flow(x_sptrain, y_sptrain,batch_size=8,seed=20031976)
print(x_sptrain.shape)
print("Image data augmentated ...")

(2746, 256, 256, 3)
Image data augmentated ...


In [36]:
import tensorflow as tf
import keras.backend as K
def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    print("test")
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    print("im here")
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [8]:
gc.collect()

0

In [9]:
effnet = DenseNet121(
        weights='imagenet',
        include_top=False,
        input_shape=(256,256,3)
)

29084464/29084464 [==============================] - 0s 0us/step


In [20]:
model = Sequential()
model.add(effnet)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dense(5, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.00005), metrics=['accuracy',f1])

In [11]:
gc.collect()

14385

In [32]:
# callback to keep track of kappa score during training
from keras.callbacks import Callback,ModelCheckpoint
from sklearn.metrics import cohen_kappa_score
import json
class KappaMetrics(Callback):
    i=0
    def on_train_begin(self, logs={}):
        self.val_kappas = []

    def on_epoch_end(self, epoch, logs={}):
        X_val, y_val = self.validation_data[:2]
        y_val = y_val.sum(axis=1) - 1

        y_pred = self.model.predict(X_val) > 0.5
        y_pred = y_pred.astype(int).sum(axis=1) - 1

        _val_kappa = cohen_kappa_score(
            y_val,
            y_pred,
            weights='quadratic'
        )

        self.val_kappas.append(_val_kappa)

        print(f"Epoch: {epoch+1} val_kappa: {_val_kappa:.4f}")
        if _val_kappa == max(self.val_kappas):
            print("Validation Kappa has improved. Saving model. - "+str(KappaMetrics.i))
            model_json = self.model.to_json()
            with open("model_in_json"+str(KappaMetrics.i)+".json", "w") as json_file:
                json.dump(model_json, json_file)
            self.model.save_weights("model_weights"+str(KappaMetrics.i)+".h5")
            KappaMetrics.i+=1
        print('im here')
        return

kappa_score = KappaMetrics()

In [47]:
x_spval.size

180092928

In [52]:
history = model.fit(
    data_generator,
    validation_data=(x_spval, y_spval),
    steps_per_epoch= 3362/8,
    epochs=8,
    callbacks=[kappa_score]
)

Epoch 1/8
419/420 [============================>.] - ETA: 0s

ValueError: Unexpected result of `train_function` (Empty logs). This could be due to issues in input pipeline that resulted in an empty dataset. Otherwise, please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['Train_Loss','Validation_Loss'])

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.legend(['Train_Accuracy','Validation_Accuracy'])